# UFGsim2024infufg
> Module to handle loading, preprocessing and postprocessing of the data from the simulated world for UFG's dataset made by the AIR-UFG perception team
>
> Still needs proper documentation

In [ ]:
#| default_exp UFGsim2024infufg

In [ ]:
#| exports
import torch
import re
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn
import yaml
from pathlib import Path
import numpy as np
from lightning import LightningDataModule
from torchvision.transforms import v2
from colorcloud.behley2019iccv import SphericalProjection

## Introduction
This is a conversion tool of the simulated dataset for UFG's autonomous driving perception software project as a pytorch [Dataset](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).

Some important notes:
1. The Dataset:
   > - Currently there is 13 labels representing vehicles, buildings and vegetation. You can check them in the yaml file.
   > - Since the dataset is built in a simulated world, the point clouds are converted from rosbags into PCD files and then from PCD to '.bin' files for proper pre-processing. In case you want to know how this was done, the tool used for the conversion can be found in the [Point Cloud Convertion Tools](https://github.com/AIR-UFG/pointcloud_conversion_tools) repository.
   > - The current lidar shape used for the Dataset construction is (h,w) = (16x440).
2. The dataset files **must** be organized as follows:
    - data
      - laser_scans
          - laser0
            - pcd1.bin
            - pcd2.bin
            - ...
          - laser1
            - pcd1.bin
            - pcd2.bin
            - ...
          - ...
      - ufg-sim.yaml


In [ ]:
#| exports
class UFGSimDataset(Dataset):
    "Load the UFGSim dataset ina pytorch Dataset object."
    def __init__(self, data_path, split='train', transform=None):
        data_path = Path(data_path)
        yaml_path = data_path/'ufg-sim.yaml'
        self.ufgsim_velodyne_path = data_path/'laser_scans'

        with open(yaml_path, 'r') as file:
            metadata = yaml.safe_load(file)

        lasers = metadata['split'][split]
        self.labels_dict = metadata['labels']

        ufgsim_velodyne_fns = []
        for laser in lasers:
            ufgsim_velodyne_fns += list(self.ufgsim_velodyne_path.rglob(f'laser{laser}/*.bin'))
  
        self.frame_ids = [fn.stem for fn in sorted(ufgsim_velodyne_fns)]
        self.frame_lasers = [fn.parts[-2] for fn in sorted(ufgsim_velodyne_fns)]

        self.content = metadata['content']
        max_key = sorted(self.content.keys())[-1]
        self.content_np = np.zeros((max_key+1,), dtype=np.float32)
        for k, v in self.content.items():
            self.content_np[k] = v

        self.learning_map = metadata['learning_map']
        max_key = sorted(self.learning_map.keys())[-1]
        self.learning_map_np = np.zeros((max_key+1,), dtype=int)
        for k, v in self.learning_map.items():
            self.learning_map_np[k] = v
        
        self.learning_map_inv = metadata['learning_map_inv']
        self.learning_map_inv_np = np.zeros((len(self.learning_map_inv),))
        self.content_sum_np = np.zeros_like(self.learning_map_inv_np, dtype=np.float32)
        for k, v in self.learning_map_inv.items():
            self.learning_map_inv_np[k] = v
            self.content_sum_np[k] = self.content_np[self.learning_map_np == k].sum()
        
        self.color_map_bgr = metadata['color_map']
        max_key = sorted(self.color_map_bgr.keys())[-1]
        self.color_map_rgb_np = np.zeros((max_key+1,3))
        for k,v in self.color_map_bgr.items():
            self.color_map_rgb_np[k] = np.array(v[::-1], np.float32)
        
        self.transform = transform
        self.is_test = (split == 'test')


    def learning_remap(self, remapping_rules):
        new_map_np = np.zeros_like(self.learning_map_np, dtype=int)
        max_key = sorted(remapping_rules.values())[-1]
        new_map_inv_np = np.zeros((max_key+1,), dtype=int)
        for k, v in remapping_rules.items():
            new_map_np[self.learning_map_np == k] = v
            if new_map_inv_np[v] == 0:
                new_map_inv_np[v] = self.learning_map_inv_np[k]
        self.learning_map_np = new_map_np
        self.learning_map_inv_np = new_map_inv_np
    
    def __len__(self):
        return len(self.frame_ids)

    def set_transform(self, transform):
        self.transform = transform
        
    def __getitem__(self, idx):
        frame_id = self.frame_ids[idx]
        frame_laser = self.frame_lasers[idx]

        frame_path = self.ufgsim_velodyne_path/frame_laser/(frame_id + '.bin')

        with open(frame_path, 'rb') as f:
            frame = np.fromfile(f, dtype=np.float32).reshape(-1, 4)
            x_frame = frame[:, 0]
            y_frame = frame[:, 1]
            z_frame = frame[:, 2]
            label = frame[:, 3].astype(np.uint32)

        label = self.learning_map_np[label]
        mask = label != 0
        weight = 1./self.content_sum_np[label]

        item = {
            'frame': frame,
            'label': label,
            'mask': mask,
            'weight': weight
        }
        if self.transform:
            item = self.transform(item)
        # return x_frame, y_frame, z_frame, label # in case of separated training 
        return item
        


Similar to the [semantic-kitti-api](https://github.com/PRBonn/semantic-kitti-api) method, we use a yaml file for label description and annotation as folllows:
> - `labels`: dictionary which maps numeric labels [...] to a string class. Example: `10: "car"`
> - `color_map`: dictionary which maps numeric labels [...] to a bgr color for visualization. Example `10: [245, 150, 100] # car, blue-ish`
> - `learning_map`: dictionary which maps each class label to its cross entropy equivalent, for learning. This is done to mask undesired classes, map different classes together, and because the cross entropy expects a value in [0, numclasses - 1]. [...] Examples:
>   ```yaml
>     0 : 0     # "unlabeled"
>     1 : 0     # "outlier" to "unlabeled" -> gets ignored in training, with unlabeled
>     10: 1     # "car"
>     252: 1    # "moving-car" to "car" -> gets merged with static car class
>   ```  
> - `learning_map_inv`: dictionary with inverse of the previous mapping [...] (for saving point cloud predictions in original label format [and to use the color_map dictionary for visualization]). [...]
> - `split`: contains 3 lists, with the sequence numbers for training, validation, and evaluation.

Lastly, use the following code to get the training data in its original format:

In [ ]:
#| eval: false
data_path = '../UFGSim/'
ds = UFGSimDataset(data_path)
test_ds = UFGSimDataset(data_path, split='test')
val_ds = UFGSimDataset(data_path, split='valid')
print(f'train size:\t{len(ds)}\ntest size:\t{len(test_ds)}\nvalid size:\t{len(val_ds)}')

In case you added/removed sequences, you'll want to update the `content` field in the __yaml__ file. In order to do that see the following cells:

In [ ]:
#| export
def calculate_frequencies(dataset):
    class_frequencies = {i: 0 for i in range(0, 13)}
    
    for item in dataset:
        labels = item['label']
        # Flatten the label array to count occurrences
        flattened_labels = labels.flatten()
        # Count the occurrences of each class
        unique, counts = np.unique(flattened_labels, return_counts=True)
        # Update the frequency dictionary
        for cls, count in zip(unique, counts):
            class_frequencies[cls] += count

    # change cases where frequency is 0 to 1
    for key, item in class_frequencies.items():
        if item == 0:
            class_frequencies[key] = 1

    class_frequencies = list(class_frequencies.values())
    return class_frequencies

In [ ]:
#| eval: false
train_freq = calculate_frequencies(ds)
test_freq = calculate_frequencies(test_ds)

In [ ]:
#| eval: false
combined_freq = [v1 + v2 for v1,v2 in zip(train_freq,test_freq)]
combined_freq

In [ ]:
#| eval: false
train_pcd = [item['frame'].shape[0] * item['frame'].shape[1] for item in ds]
test_pcd = [item['frame'].shape[0] * item['frame'].shape[1] for item in test_ds]
combined_pcd = sum(train_pcd) + sum(test_pcd)
combined_pcd

In [ ]:
#| eval: false
content = [v1 / combined_pcd for v1 in combined_freq]
content

Now, check if the dictionary keys are as expected:

In [ ]:
#| eval: false
item = ds[128]
frame = item['frame']
frame[:,3], frame.shape

In [ ]:
#| eval: false
label = item['label']
label, label.shape, label.dtype, set(label)

In [ ]:
#| eval: false
mask = item['mask']
mask, mask.shape, mask.dtype

In [ ]:
#| eval: false
weights = item['weight']
weights, weights.shape, weights.dtype

## Lidar Range Image Projections
In order to process the 3D LiDAR data more efficiently, there is a famous approach to convert the 3D data into 2D data, and in the case of a rotating LiDAR scan, all point can be projected to an image using a process called **spherical projection**. 

This concept can be found in the following papers categorized by the projection algorithm and ordered chronologically:
> - [SqueezeSeg: Convolutional Neural Nets with Recurrent CRF for Real-Time Road-Object Segmentation from 3D LiDAR Point Cloud](https://arxiv.org/pdf/1710.07368)
> - [Range-Image: Incorporating sensor topology for LiDAR point cloud processing](https://hal.science/hal-01756975/document)
> - [RIU-Net: Embarrassingly simple semantic segmentation of 3D LiDAR point cloud](https://arxiv.org/pdf/1905.08748)

The code for this convertion as further explanations and inspirations used in this work can be seen in the [behley2019iccv pytorch module](https://github.com/AIR-UFG/colorcloud/blob/main/nbs/00_behley2019iccv.ipynb) also made by AIR-UFG.

In [ ]:
#| exports
class ProjectionSimTransform(nn.Module):
    def __init__(self, projection):
        super().__init__()
        self.projection = projection
        self.W = projection.W
        self.H = projection.H

    def forward(self, item):
        frame = item['frame']
        label = item['label']
        mask = item['mask']
       # weight = item['weight']

        scan_xyz = frame[:,:3]
        depth = np.linalg.norm(scan_xyz, 2, axis=1)

        proj_x, proj_y, outliers = self.projection.get_xy_projections(scan_xyz, depth)

        # filter outliers
        if outliers is not None:
            proj_x = proj_x[~outliers]
            proj_y = proj_y[~outliers]
            scan_xyz = scan_xyz[~outliers]
            depth = depth[~outliers]
            if label is not None:
                label = label[~outliers]
                mask = mask[~outliers]
               # weight = weight[~outliers]

        order = np.argsort(depth)[::-1]
        info_list = [scan_xyz, depth[..., np.newaxis]]
        if label is not None:
           # info_list += [weight[..., np.newaxis]]
            info_list += [mask[..., np.newaxis]]
            info_list += [label[..., np.newaxis]]

        scan_info = np.concatenate(info_list, axis=-1)
        scan_info = scan_info[order]
        proj_y = proj_y[order]
        proj_x = proj_x[order]


        projections_img = np.zeros((self.H, self.W, 2+len(info_list)), dtype=np.float32)
        projections_img[:,:,-1] -= 1
        projections_img[proj_y, proj_x] = scan_info

        if label is not None:
            frame_img = projections_img[:,:,:-2]
            label_img = projections_img[:,:,-1].astype(int)
            mask_img = projections_img[:,:,-2].astype(bool)
            mask_img = mask_img & (label_img > -1)
           # weight_img = projections_img[:,:,-3]

        else:
            frame_img = projections_img
            label_img = None
            mask_img = projections_img[:,:,-1] >= 0
           # weight_img = None

        item = {
            'frame': frame_img,
            'label': label_img,
            'mask': mask_img,
            #'weight': weight_img,
        }
        
        return item

With the projection algorithm described previously, we can construct a `ProjectionSimTransform` object that can be used with the `UFGSimDataset`. Here is an example on how to use it:

In [ ]:
#| eval: false
proj = SphericalProjection(fov_up_deg=15., fov_down_deg=-15., W=440, H=16)
tfms = ProjectionSimTransform(proj)
ds.set_transform(tfms)
item = ds[128]

As we can see below, the data is still returned as numpy arrays, but their shapes have changed as follows:

- frame: from a sequence of 4D float vectors (i.e. x, y, z, labels) to an image of HxW resolution with 4 channels with float numbers (i.e. x, y, z, depth);
- label: from a sequence of integers to an image of HxW resolution with a single channel with an integer;
- mask: from a sequence of booleans to an image of HxW resolution with a single channel with booleans indicating which pixels should be ignored during training and inference.

There are a lot of pixels in these projection images that have no corresponding LiDAR readings to be projected. Hence, these pixels were filled with:

- in frame_img: 0.
- in label_img: -1

The mask image provides a convenient boolean mask to easily identify pixels with valid LiDAR readings marked as 'True' and pixels with no LiDAR readings marked as 'False'.

In [ ]:
#| eval: false
frame_img = item['frame']
np.unique(frame_img[:2]), shframe_img.ape, frame_img.dtype

In [ ]:
#| eval: false
label_img = item['label']
label_img[:2], label_img.shape, label_img.dtype, set(label_img.flatten())

In [ ]:
#| eval: false
mask_img = item['mask']
mask_img[:2], mask_img.shape, mask_img.dtype

In [ ]:
#| exports
class ProjectionSimVizTransform(nn.Module):
    def __init__(self, color_map_rgb_np, learning_map_inv_np):
        super().__init__()
        self.color_map_rgb_np = color_map_rgb_np
        self.learning_map_inv_np = learning_map_inv_np

    def scale(self, img, min_value, max_value):
        img = img.clip(min_value, max_value)
        return (255.*(img - min_value)/(max_value - min_value)).astype(int)

    def forward(self, item):
        frame_img = item['frame']
        label_img = item['label']
        mask_img = item['mask']
        #weight_img = item['weight']
        
        normalized_frame_img = None
        if frame_img is not None:
            x = self.scale(frame_img[:,:,0], -100., 100.)
            y = self.scale(frame_img[:,:,1], -100., 100.)
            z = self.scale(frame_img[:,:,2], -31., 5.)
            d = self.scale(frame_img[:,:,3], 0., 100.)
            normalized_frame_img = np.stack((x, y, z, d), axis=-1)
            normalized_frame_img[mask_img == False] *= 0

        colored_label_img = None
        if label_img is not None:
            label_img[mask_img] = self.learning_map_inv_np[label_img[mask_img]]
            colored_label_img = np.zeros(label_img.shape + (3,))
            colored_label_img[mask_img] = self.color_map_rgb_np[label_img[mask_img]]
            colored_label_img = colored_label_img.astype(int)

        item = {
            'frame': normalized_frame_img,
            'label': colored_label_img,
            'mask': mask_img,
            #'weight': weight_img,          
        }
        return item

In order to plot comparable visualizations of the projections, we need to standardize the scale of the float channels from the frame projection and map the label integers to their corresponding RGB color defined in the `color_map` dict from the yaml file that was downloaded before. After some experimentation with several samples from the entire dataset, the hardcoded scale values in the `ProjectionSimVizTransform.forward` method seemed to work reasonably well.

Here is an example on how to use and compose the previous transforms with the `torchvision.transforms.v2` module, and then visualize the resulting images with the `matplolib.pyplot` module:

In [ ]:
#| eval: false
from matplotlib import pyplot as plt

In [ ]:
#| exports
def plot_projections(img, label, channels=['x', 'y', 'z', 'r', 'd'], channels_map = {"x": 0, "y": 1, "z": 2, "r": 3, "d": 4}):
    num_channels = len(channels)
    fig_size_vertical = 2*num_channels
    fig, axs = plt.subplots(num_channels + 1, 1, figsize=(20, fig_size_vertical))
    
    for i, (ax, title) in enumerate(zip(axs, channels + ['label'])):
        if i < num_channels:
            j = channels_map[channels[i]]
            ax.imshow(img[:, :, j])
        else:
            ax.imshow(label)
        ax.set_title(title)
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
#| eval: false
proj = SphericalProjection(fov_up_deg=15., fov_down_deg=-15., W=440, H=16) 
tfms = v2.Compose([
    ProjectionSimTransform(proj),
    ProjectionSimVizTransform(ds.color_map_rgb_np, ds.learning_map_inv_np),
])
ds.set_transform(tfms)
item = ds[128]

img = item['frame']
label = item['label']
channels_map = {"x": 0, "y": 1, "z": 2, "d": 3}

plot_projections(img, label, channels=['x', 'y', 'z', 'd'], channels_map=channels_map)

In the next cell, we can see how long it takes on average to sample an item from this dataset class.

In [ ]:
#| eval: false
%timeit item = ds[0]

In [ ]:
#| exports
class ProjectionToTensorTransformSim(nn.Module):
    "Pytorch transform that converts the projections from np.array to torch.tensor. It also changes the frame image format from (H, W, C) to (C, H, W)."
    def forward(self, item):
        frame_img = item['frame']
        label_img = item['label']
        mask_img = item['mask']
       # weight_img = item['weight']
        
        frame_img = np.transpose(frame_img, (2, 0, 1))
        frame_img = torch.from_numpy(frame_img).float()
        if label_img is not None:
            label_img = torch.from_numpy(label_img.astype(np.int64))
          #  weight_img = torch.from_numpy(weight_img)
        if mask_img is not None:
            mask_img = torch.from_numpy(mask_img)
        
        item = {
            'frame': frame_img,
            'label': label_img,
            'mask': mask_img,
            #'weight': weight_img,
        }
        return item

In order to use the dataset to train pytorch models, it is necessary to transform the data from `np.array` to `torch.tensor`. Lastly, for the frame images, it is a must to transpose/permute its axis from channels **last** to channels **first**: (H,W,C) -> (C, H, W).

Here is an example on how to use it with the scan spherical projection:

In [ ]:
#| eval: false
proj = SphericalProjection(fov_up_deg=15., fov_down_deg=-15., W=440, H=16)
tfms = v2.Compose([
    ProjectionSimTransform(proj),
    ProjectionToTensorTransformSim(),
])
ds.set_transform(tfms)
item = ds[127]
img = item['frame']
label = item['label']
mask = item['mask']
#weight = item['weight']

print(img.shape, img.type())
print(label.shape, label.dtype)
print(mask.shape, mask.type())
#print(weight.shape, weight.type())

Here is an example on how to combine the classes above to implement a `torch.utils.data.Dataloader` that iterates on batches of frame images, labels and masks:

In [ ]:
#| eval: false
bs = 10
dl = DataLoader(ds, bs, num_workers=0)
for batch in dl:
    item = batch
    img = item['frame']
    label = item['label']
    mask = item['mask']
   # weight = item['weight']
    print(f"Shape of img \t [N, C, H, W]: \t {img.shape}")
    print(f"Shape of label \t [N, H, W]: \t {label.shape}")
    print(f"Shape of mask \t [N, H, W]: \t {mask.shape}")
   # print(f"Shape of weight \t [N, H, W]: \t {weight.shape}")
    break

In the next cell, we can see how long it takes on average to iterate the dataloader for 1 epoch.

In [ ]:
#| eval: false
from tqdm.auto import tqdm

In [ ]:
#| eval: false
for batch in tqdm(dl):
    item = batch

In [ ]:
#| eval: false
ds = UFGSimDataset(data_path)
proj = SphericalProjection(fov_up_deg=15., fov_down_deg=-15., W=440, H=16)
tfm = ProjectionSimTransform(proj)
ds.set_transform(tfm)

bs = 1000
dl = DataLoader(ds, bs)
item = next(iter(dl))
img = item['frame']
label = item['label']
mask = item['mask']

idx = ((label <= 0).sum(axis=1) > 60).sum(axis=1).argmax()

tfms = v2.Compose([
    ProjectionSimTransform(proj),
    ProjectionSimVizTransform(ds.color_map_rgb_np, ds.learning_map_inv_np),
])
ds.set_transform(tfms)

item = ds[idx]
i = item['frame']
l = item['label']
channels_map = {"x": 0, "y": 1, "z": 2, "d": 3}
plot_projections(i, l, channels=['x', 'y', 'z', 'd'], channels_map=channels_map)

In [ ]:
#| eval: false
ds.set_transform(tfm)

bs = 1000
dl = DataLoader(ds, bs, shuffle=True)
item = next(iter(dl))
img = item['frame']
label = item['label']
mask = item['mask']

sparsity = (label <= 0).sum(axis=(1,2))/label[0].numel()
plt.hist(sparsity, bins=100);

In [ ]:
#| eval: false
l = label.flatten()
values, counts = np.unique(l, return_counts=True)
plt.bar(values, np.log10(counts));

## Reversing the projection

Since this work is meant to be used to train an autonomous car in a simulated world to serve as validation for the real world training and the training of the LiDAR 3D data is done by projecting into an image, it is necessary to reverse the projection (2D projected image to 3D LiDAR point cloud) of the training in order to be used as data for the car.

In [ ]:
#| exports
class ProjectionSimToPointCloud(nn.Module):
    def __init__(self, proj_fov_up=15.0, proj_fov_down=-15.0):
        super().__init__()
        self.proj_fov_up = proj_fov_up
        self.proj_fov_down = proj_fov_down
        self.fov_up = self.proj_fov_up / 180.0 * np.pi
        self.fov_down = self.proj_fov_down / 180.0 * np.pi

    def forward(self, image):
        h, w = image[:,:,3].shape
        points = []

        for i in range(h):
            for j in range(w):
                pitch = (1 - i/h) * (self.fov_up + abs(self.fov_down)) - abs(self.fov_down)
                yaw = ((j / w) * np.pi * 2) - np.pi
                depth = image[:,:,3][i][j]

                x = depth * np.cos(yaw)
                y = -depth * np.sin(yaw)
                z = depth * np.sin(pitch)
                points.append([x, y, z])

        points = np.array(points)
        return torch.tensor(points, dtype=torch.float32)

Notice that in order to do this process, the `depth` value is needed. This is because the depth value it obtained by the (x,y,z) values. That being said, inverting the `spherical projection` equations and with some trigonometry applied to the depth value, it should be possible to retrieve the original point cloud data.

Here is an example of how to use it:

In [ ]:
#| eval: false
ds = UFGSimDataset(data_path)
frame, _, _ = ds[127]
print(f'Original point cloud:\t{frame.shape}')
proj = SphericalProjection(fov_up_deg=15., fov_down_deg=-15., W=440, H=16)
tfm = ProjectionSimTransform(proj)
ds.set_transform(tfm)
frame, _, _ = ds[127]
print(f'Projected image:\t{frame.shape}')
b2p = ProjectionSimToPointCloud()
pcd = b2p(frame)
print(f'Reprojected to point cloud:\t{pcd.shape}')

## LightningDataModules for benchmarking

When benchmarking different semantic segmentation algorithms, the following LightningDataModule should be used.

In [ ]:
#| exports
class SemanticSegmentationSimLDM(LightningDataModule):
    "Lightning DataModule to facilitate reproducibility of experiments."
    def __init__(self, 
                 proj_style='spherical',
                 proj_kargs={'fov_up_deg': 15.,'fov_down_deg': -15., 'W': 440, 'H': 16,},
                 train_batch_size=8, 
                 eval_batch_size=16,
                 num_workers=8
                ):
        super().__init__()

        proj_class = {
        #'unfold': UnfoldingProjection,
        'spherical': SphericalProjection
        }
        assert proj_style in proj_class.keys()
        self.proj = proj_class[proj_style](**proj_kargs)
        self.train_batch_size = train_batch_size
        self.eval_batch_size = eval_batch_size
        self.num_workers = num_workers
    
    def setup(self, stage: str):
        data_path = '../UFGSim/'
        tfms = v2.Compose([
            ProjectionSimTransform(self.proj),
            ProjectionToTensorTransformSim(),
        ])
        split = stage
        if stage == 'fit':
            split = 'train'
        else:
            split = 'test'
        
        ds = UFGSimDataset(data_path, split, transform=tfms)
        if not hasattr(self, 'viz_tfm'):
            self.viz_tfm = ProjectionSimVizTransform(ds.color_map_rgb_np, ds.learning_map_inv_np)
        
        if stage == "fit":
            self.ds_train = ds
            self.ds_val = UFGSimDataset(data_path, 'valid', tfms)
        
        if stage == "test":
            self.ds_test = ds
        if stage == "predict":
            self.ds_predict = ds
            

    def train_dataloader(self):
        return DataLoader(self.ds_train, batch_size=self.train_batch_size, num_workers=self.num_workers, shuffle=True, drop_last=True)

    def val_dataloader(self):
        return DataLoader(self.ds_val, batch_size=self.eval_batch_size, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.ds_test, batch_size=self.eval_batch_size, num_workers=self.num_workers)

    def predict_dataloader(self):
        return DataLoader(self.ds_predict, batch_size=self.eval_batch_size, num_workers=self.num_workers)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()